# Capability — Single-Image In-Context Learning
Guide the model with one exemplar image before detecting the same object in a different scene.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ericpence/perceptron_repo/blob/main/cookbook/recipes/capabilities/in-context-learning/in-context-learning.ipynb)

## Install dependencies
Install the SDK plus helpers for drawing overlays inside the notebook session.

In [ ]:
%pip install --upgrade perceptron pillow --quiet

## Configure the Perceptron client
Load the API key (inline or from the environment) once, then reuse the configured client for the rest of the notebook.

In [ ]:
from pathlib import Path

from IPython.display import Image as IPyImage
from IPython.display import display
from PIL import Image, ImageDraw, ImageFont

from cookbook.utils import cookbook_asset
from perceptron import annotate_image, bbox, configure, detect

# configure() reads PERCEPTRON_API_KEY from the environment.
# configure() reads PERCEPTRON_API_KEY from the environment.
configure(
    provider="perceptron",
    # model="isaac-0.1",  # Enable once the SDK supports the model argument.
)

EXAMPLE_IMAGE = cookbook_asset("in-context-learning", "single", "cake_mixer_example.webp")
TARGET_IMAGE = cookbook_asset("in-context-learning", "single", "find_kitchen_item.webp")
ANNOTATED_PATH = Path("find_kitchen_item_annotated.png")

for path in (EXAMPLE_IMAGE, TARGET_IMAGE):
    if not path.exists():
        raise FileNotFoundError(f"Missing asset: {path}")

## Bootstrap the exemplar box
Run one detection pass on the exemplar image so we can feed a precise bounding box back as context.

In [ ]:
bootstrap = detect(
    str(EXAMPLE_IMAGE),
    classes=["objectCategory1"],
    max_outputs=1,
)
if not bootstrap.points:
    raise RuntimeError("Detect returned no boxes for the exemplar. Adjust the label or image.")

first_box = bootstrap.points[0]
example_shot = annotate_image(
    str(EXAMPLE_IMAGE),
    {
        "objectCategory1": [
            bbox(
                int(first_box.top_left.x),
                int(first_box.top_left.y),
                int(first_box.bottom_right.x),
                int(first_box.bottom_right.y),
                mention="objectCategory1",
            )
        ]
    },
)
collections = example_shot.get("collections") or []
boxes = example_shot.get("boxes") or []
placeholder = "objectCategory1"
if collections:
    exemplar_annotation = collections[0]
elif boxes:
    exemplar_annotation = boxes[0]
else:
    raise RuntimeError("annotate_image returned no collections or boxes; check the exemplar annotations.")
print("Prepared exemplar guidance with", getattr(exemplar_annotation, "mention", placeholder) or placeholder)

## Detect the same object in a new scene
Pass the exemplar annotation back to `detect` via the `examples` argument to nudge the model toward consistent grounding.

In [ ]:
result = detect(
    str(TARGET_IMAGE),
    classes=["objectCategory1"],
    examples=[example_shot],
)

print(result.text)
boxes = result.points or []
print(f"Returned {len(boxes)} grounded regions")

## Render the grounded output
Overlay the predicted boxes on the target scene and preview the saved PNG inline.

In [ ]:
img = Image.open(TARGET_IMAGE).convert("RGB")
draw = ImageDraw.Draw(img)
try:
    font = ImageFont.truetype("arial.ttf", size=20)
except OSError:
    font = ImageFont.load_default()

if boxes:

    def to_px(point):
        return point.x / 1000 * img.width, point.y / 1000 * img.height

    for box in boxes:
        top_left = to_px(box.top_left)
        bottom_right = to_px(box.bottom_right)
        draw.rectangle([top_left, bottom_right], outline="lime", width=3)
        label = box.mention or getattr(box, "label", None) or "objectCategory1"
        text_position = (top_left[0], max(top_left[1] - 20, 0))
        draw.text(text_position, label, fill="lime", font=font)
else:
    print("No boxes returned; adjust the exemplar or label and rerun.")

img.save(ANNOTATED_PATH)
display(IPyImage(filename=str(ANNOTATED_PATH)))
print(f"Saved annotated target to {ANNOTATED_PATH}")

## Conclusion & next steps
- Swap in different exemplar / target pairs to explore other categories.
- Add more than one exemplar by appending to the `examples` list for tougher distinctions.
- Combine this flow with detection or Q&A helpers to build end-to-end pipelines.